In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "2"

In [2]:
import pickle
from seq2struct.models.match_model import MatchModel

W0913 06:56:35.580819 140373184734976 deprecation_wrapper.py:119] From /home/kavigupta/anaconda3/envs/py36/lib/python3.6/site-packages/bert/optimization.py:87: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



W0913 06:56:40.211593 140373184734976 deprecation_wrapper.py:119] From /srv/home/kavigupta/seq2struct/seq2struct/models/match_model.py:40: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

W0913 06:56:42.277941 140373184734976 deprecation_wrapper.py:119] From /home/kavigupta/anaconda3/envs/py36/lib/python3.6/site-packages/bert/tokenization.py:125: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.



W0913 06:56:45.769739 140373184734976 deprecation_wrapper.py:119] From /home/kavigupta/anaconda3/envs/py36/lib/python3.6/site-packages/bert/run_classifier.py:774: The name tf.logging.info is deprecated. Please use tf.compat.v1.logging.info instead.



NameError: name 'num_train_steps' is not defined

In [39]:
df_cm = pd.DataFrame(
    np.array([
        [confusion["true_positives"], confusion["false_negatives"]],
        [confusion["false_positives"], confusion["true_negatives"]],
    ], dtype=np.int),
    index = list("+-"),
    columns = [i for i in list("+-")])

In [70]:
getPrediction(
    ["What are the names, countries, and ages for every singer in descending order of age?",
    "What are the names, countries, and ages for every singer in descending order of age?"],
    ["SELECT singer.Name, singer.Country, singer.Age FROM singer ORDER BY singer.Age Desc",
     "SELECT singer.Name, singer.Country, singer.Country FROM singer ORDER BY singer.Age Desc"])

[('What are the names, countries, and ages for every singer in descending order of age?',
  'SELECT singer.Name, singer.Country, singer.Age FROM singer ORDER BY singer.Age Desc',
  array([-5.030763  , -0.00655524], dtype=float32),
  1),
 ('What are the names, countries, and ages for every singer in descending order of age?',
  'SELECT singer.Name, singer.Country, singer.Country FROM singer ORDER BY singer.Age Desc',
  array([-0.1063258, -2.293939 ], dtype=float32),
  0)]